# 생성형 AI로 게임 에셋 만들기

이 교안에서는 **생성형 AI(Generative AI)**를 사용해서 게임에 필요한 이미지 에셋을 만드는 방법을 배웁니다.

## 목차
1. 생성형 AI란?
2. API 호출 기초
3. OpenAI DALL-E로 이미지 생성
4. Stability AI로 이미지 생성
5. 생성된 이미지를 게임 에셋으로 가공
6. 실습 과제

---
## 1. 생성형 AI란?

### 기존 AI vs 생성형 AI

| 구분 | 기존 AI | 생성형 AI |
|------|---------|----------|
| 하는 일 | 분류, 예측 (이 사진은 고양이다) | 새로운 콘텐츠 생성 (고양이 그림을 그려줘) |
| 입력 | 데이터 | 텍스트 프롬프트 (자연어) |
| 출력 | 라벨, 숫자 | 이미지, 텍스트, 음악, 코드 |

### 대표적인 이미지 생성 모델

- **DALL-E** (OpenAI): ChatGPT 만든 회사의 이미지 생성 모델. 텍스트 설명을 넣으면 이미지를 만들어줌
- **Stable Diffusion** (Stability AI): 오픈소스 기반 이미지 생성 모델. 커스터마이징 가능
- **Midjourney**: 디스코드 기반 이미지 생성 서비스. 예술적 품질이 높음

### 게임 에셋 제작에서의 활용

우리 CG Town 프로젝트에서는 이런 에셋이 필요합니다:
- 타일맵 타일 (잔디, 물, 길 등) - **64x64 픽셀**
- 오브젝트 (책상, 의자, 건물 등)
- 캐릭터 스프라이트

생성형 AI를 사용하면 디자이너 없이도 이런 에셋을 빠르게 만들 수 있습니다!

---
## 2. API 호출 기초

### API란?

**API (Application Programming Interface)**는 프로그램끼리 대화하는 방법입니다.

쉽게 말하면:
- 우리가 카페에서 주문할 때: "아메리카노 한 잔 주세요" → 커피가 나옴
- 프로그램이 API를 호출할 때: "고양이 그림 그려줘" → 이미지가 나옴

### HTTP 요청의 기본

웹에서 API를 호출할 때는 **HTTP 요청**을 보냅니다:

| 메서드 | 용도 | 예시 |
|--------|------|------|
| GET | 데이터 가져오기 | 날씨 정보 조회 |
| POST | 데이터 보내기 | 이미지 생성 요청 |

### API 키란?

API를 사용하려면 **API 키**가 필요합니다. 이건 일종의 **신분증** 같은 거예요.
- 누가 요청했는지 확인하고
- 사용량을 추적하고
- 요금을 부과하는 데 쓰입니다

### 실습: 먼저 필요한 라이브러리를 설치합니다

아래 셀을 실행하세요. (이미 설치되어 있다면 건너뛰어도 됩니다)

In [ ]:
# 필요한 라이브러리 설치
!pip install openai requests Pillow python-dotenv matplotlib

### API 키 안전하게 관리하기

API 키는 **절대 코드에 직접 적으면 안 됩니다!** (깃허브에 올라가면 큰일납니다)

대신 `.env` 파일에 저장하고, `python-dotenv`로 불러옵니다.

1. 이 노트북과 같은 폴더에 `.env` 파일을 만드세요
2. 아래 내용을 넣으세요:

```
OPENAI_API_KEY=여기에_오픈AI_키_붙여넣기
STABILITY_API_KEY=여기에_스태빌리티_키_붙여넣기
```

In [ ]:
# .env 파일에서 API 키 불러오기
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
STABILITY_API_KEY = os.getenv("STABILITY_API_KEY")

# 키가 잘 로드되었는지 확인 (앞 8자만 보여줌)
if OPENAI_API_KEY:
    print(f"OpenAI API Key: {OPENAI_API_KEY[:8]}...")
else:
    print("OpenAI API Key가 없습니다. .env 파일을 확인하세요!")

if STABILITY_API_KEY:
    print(f"Stability API Key: {STABILITY_API_KEY[:8]}...")
else:
    print("Stability API Key가 없습니다. .env 파일을 확인하세요!")

### 실습: 간단한 API 호출 해보기

먼저 공개 API로 연습합시다. API 키 없이도 호출 가능한 무료 API입니다.

In [ ]:
import requests

# 무료 공개 API 호출 예제: 랜덤 고양이 사진 가져오기
response = requests.get("https://api.thecatapi.com/v1/images/search")

print(f"상태 코드: {response.status_code}")  # 200이면 성공!
print(f"응답 데이터: {response.json()}")

# 이미지 URL 추출
cat_image_url = response.json()[0]["url"]
print(f"\n고양이 이미지 URL: {cat_image_url}")

In [ ]:
# 가져온 이미지를 Jupyter에서 바로 보기
from IPython.display import Image, display

# URL에서 이미지 다운로드
img_data = requests.get(cat_image_url).content

# 화면에 표시
display(Image(data=img_data, width=300))
print("API로 가져온 고양이 사진입니다!")

### 정리

지금까지 배운 것:
1. `requests.get(URL)` → 데이터 가져오기 (GET 요청)
2. `response.status_code` → 성공(200) / 실패 확인
3. `response.json()` → JSON 형태로 응답 데이터 읽기

이제 이 방식으로 AI 이미지 생성 API를 호출해 봅시다!

---
## 3. OpenAI DALL-E로 이미지 생성

### OpenAI API 키 발급 방법

1. https://platform.openai.com 접속
2. 회원가입 / 로그인
3. 좌측 메뉴 → **API keys** 클릭
4. **Create new secret key** 클릭
5. 생성된 키를 `.env` 파일의 `OPENAI_API_KEY`에 붙여넣기

### 가격
- DALL-E 3: 이미지 1장당 약 $0.04 ~ $0.12
- 신규 가입 시 $5 무료 크레딧 (약 100장 이상 생성 가능)

In [ ]:
from openai import OpenAI

# OpenAI 클라이언트 생성
client = OpenAI(api_key=OPENAI_API_KEY)

# DALL-E로 이미지 생성!
response = client.images.generate(
    model="dall-e-3",
    prompt="A cute pixel art office desk with a computer monitor, keyboard, and coffee cup. Top-down view, 2D game asset style, clean background",
    size="1024x1024",
    quality="standard",
    n=1,  # 1장 생성
)

# 생성된 이미지 URL
image_url = response.data[0].url
print(f"생성된 이미지 URL: {image_url[:80]}...")

In [ ]:
# 생성된 이미지 확인하기
from IPython.display import Image, display

img_data = requests.get(image_url).content
display(Image(data=img_data, width=400))
print("DALL-E가 생성한 이미지입니다!")

In [ ]:
# 생성된 이미지를 파일로 저장하기
output_path = "outputs/dalle_office_desk.png"

with open(output_path, "wb") as f:
    f.write(img_data)

print(f"이미지가 {output_path}에 저장되었습니다!")

### 게임 에셋용 프롬프트 작성 팁

좋은 프롬프트를 쓰면 더 좋은 결과가 나옵니다!

**기본 구조**: `[스타일] + [시점] + [대상] + [배경] + [용도]`

**예시 프롬프트들**:

| 용도 | 프롬프트 |
|------|----------|
| 잔디 타일 | `pixel art grass tile, top-down view, seamless tileable texture, 2D game asset, green natural grass` |
| 사무실 책상 | `pixel art office desk with computer, top-down view, 2D game asset, clean transparent background` |
| 건물 | `pixel art small office building, top-down isometric view, 2D RPG game style, detailed` |

**핵심 키워드**:
- `pixel art` - 픽셀 아트 스타일
- `top-down view` - 위에서 내려다보는 시점 (우리 게임 시점)
- `2D game asset` - 2D 게임용 에셋
- `seamless tileable` - 이음새 없이 반복 가능한 타일
- `transparent background` - 투명 배경 (오브젝트에 유용)

In [ ]:
# 직접 프롬프트를 바꿔가며 실험해 보세요!

my_prompt = "pixel art office chair, top-down view, 2D game asset, simple clean style, white background"

response = client.images.generate(
    model="dall-e-3",
    prompt=my_prompt,
    size="1024x1024",
    quality="standard",
    n=1,
)

image_url = response.data[0].url
img_data = requests.get(image_url).content

display(Image(data=img_data, width=400))

# 저장
with open("outputs/dalle_custom.png", "wb") as f:
    f.write(img_data)
print("저장 완료!")

---
## 4. Stability AI로 이미지 생성

### Stability AI API 키 발급 방법

1. https://platform.stability.ai 접속
2. 회원가입 / 로그인
3. **API Keys** 메뉴에서 키 생성
4. `.env` 파일의 `STABILITY_API_KEY`에 붙여넣기

### 가격
- 이미지 1장당 약 $0.03 ~ $0.08 (DALL-E보다 저렴)
- 신규 가입 시 25 크레딧 무료

### DALL-E와의 차이점

| 항목 | DALL-E 3 | Stability AI |
|------|----------|-------------|
| 가격 | $0.04~0.12 | $0.03~0.08 |
| 특징 | 프롬프트 이해력 높음 | 픽셀아트에 더 적합 |
| 라이브러리 | `openai` (공식) | `requests` (REST API 직접 호출) |
| 커스텀 | 제한적 | 파인튜닝 가능 |

In [ ]:
import requests
import base64
from IPython.display import Image, display

# Stability AI API 직접 호출 (requests 사용)
# DALL-E와 달리 전용 라이브러리 없이 HTTP 요청으로 직접 호출합니다

url = "https://api.stability.ai/v2beta/stable-image/generate/core"

headers = {
    "Authorization": f"Bearer {STABILITY_API_KEY}",
    "Accept": "image/*",  # 이미지를 직접 바이너리로 받음
}

# POST 요청으로 이미지 생성
response = requests.post(
    url,
    headers=headers,
    files={"none": ""},  # multipart/form-data 형식 필요
    data={
        "prompt": "pixel art office desk with computer monitor and keyboard, top-down view, 2D game asset, clean style",
        "output_format": "png",
        "aspect_ratio": "1:1",
    },
)

if response.status_code == 200:
    # 이미지 저장
    output_path = "outputs/stability_office_desk.png"
    with open(output_path, "wb") as f:
        f.write(response.content)
    
    display(Image(data=response.content, width=400))
    print(f"Stability AI 이미지 저장: {output_path}")
else:
    print(f"에러 발생! 상태 코드: {response.status_code}")
    print(response.json())

### DALL-E vs Stability AI 비교

같은 프롬프트로 두 API의 결과물을 비교해 봅시다.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import os

# 생성된 이미지가 있는 경우에만 비교 표시
dalle_path = "outputs/dalle_office_desk.png"
stability_path = "outputs/stability_office_desk.png"

images = []
titles = []

if os.path.exists(dalle_path):
    images.append(PILImage.open(dalle_path))
    titles.append("DALL-E 3")

if os.path.exists(stability_path):
    images.append(PILImage.open(stability_path))
    titles.append("Stability AI")

if images:
    fig, axes = plt.subplots(1, len(images), figsize=(6 * len(images), 6))
    if len(images) == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img)
        ax.set_title(title, fontsize=16)
        ax.axis("off")
    plt.tight_layout()
    plt.show()
else:
    print("비교할 이미지가 없습니다. 위의 셀들을 먼저 실행하세요!")

---
## 5. 생성된 이미지를 게임 에셋으로 가공

AI가 생성한 이미지는 보통 1024x1024처럼 큰 사이즈입니다.
우리 게임에서는 **64x64 픽셀** 크기가 필요하므로, 이미지를 가공해야 합니다.

### Pillow 라이브러리

**Pillow (PIL)**는 Python에서 이미지를 다루는 가장 인기 있는 라이브러리입니다.

In [ ]:
from PIL import Image as PILImage

# 이미지 기본 정보 확인
sample_path = "outputs/dalle_office_desk.png"

if os.path.exists(sample_path):
    img = PILImage.open(sample_path)
    print(f"이미지 크기: {img.size}")        # (가로, 세로) 픽셀
    print(f"이미지 모드: {img.mode}")        # RGB, RGBA 등
    print(f"이미지 포맷: {img.format}")      # PNG, JPEG 등
else:
    print(f"{sample_path} 파일이 없습니다. 위의 DALL-E 셀을 먼저 실행하세요!")

### 5-1. 이미지 리사이즈 (크기 변경)

1024x1024 이미지를 64x64로 줄여봅시다.

In [ ]:
from PIL import Image as PILImage
import os

TILE_SIZE = 64  # 우리 게임의 타일 크기

sample_path = "outputs/dalle_office_desk.png"

if os.path.exists(sample_path):
    img = PILImage.open(sample_path)

    # 64x64로 리사이즈
    # LANCZOS: 고품질 리사이징 (축소할 때 가장 깔끔)
    # NEAREST: 픽셀아트에 적합 (픽셀이 뭉개지지 않음)
    resized = img.resize((TILE_SIZE, TILE_SIZE), PILImage.Resampling.NEAREST)

    # 저장
    resized.save("outputs/desk_64x64.png")

    # 비교해서 보기
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    ax1.imshow(img)
    ax1.set_title(f"원본 ({img.size[0]}x{img.size[1]})", fontsize=14)
    ax1.axis("off")

    # 64x64는 너무 작으니까 확대해서 보여줌
    ax2.imshow(resized.resize((256, 256), PILImage.Resampling.NEAREST))
    ax2.set_title(f"리사이즈 ({TILE_SIZE}x{TILE_SIZE}, 확대 표시)", fontsize=14)
    ax2.axis("off")

    plt.tight_layout()
    plt.show()
    print("64x64 타일이 저장되었습니다: outputs/desk_64x64.png")
else:
    print(f"{sample_path} 파일이 없습니다.")

### 5-2. 이미지 크롭 (자르기)

하나의 큰 이미지에서 필요한 부분만 잘라낼 수 있습니다.

In [ ]:
from PIL import Image as PILImage
import os

sample_path = "outputs/dalle_office_desk.png"

if os.path.exists(sample_path):
    img = PILImage.open(sample_path)
    w, h = img.size

    # 이미지의 중앙 부분만 크롭
    # crop((left, top, right, bottom))
    center_crop = img.crop((
        w // 4,      # left: 전체의 1/4 지점부터
        h // 4,      # top
        w * 3 // 4,  # right: 전체의 3/4 지점까지
        h * 3 // 4,  # bottom
    ))

    # 크롭한 부분을 64x64로 리사이즈
    tile = center_crop.resize((64, 64), PILImage.Resampling.NEAREST)
    tile.save("outputs/cropped_tile_64x64.png")

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

    ax1.imshow(img)
    ax1.set_title("원본", fontsize=14)
    ax1.axis("off")

    ax2.imshow(center_crop)
    ax2.set_title("중앙 크롭", fontsize=14)
    ax2.axis("off")

    ax3.imshow(tile.resize((256, 256), PILImage.Resampling.NEAREST))
    ax3.set_title("64x64 타일 (확대)", fontsize=14)
    ax3.axis("off")

    plt.tight_layout()
    plt.show()
else:
    print(f"{sample_path} 파일이 없습니다.")

### 5-3. 큰 이미지를 여러 타일로 분할하기

AI가 생성한 큰 이미지(예: 타일셋)를 64x64 타일 여러 장으로 분할하는 방법입니다.

In [ ]:
from PIL import Image as PILImage
import os

def split_into_tiles(image_path, tile_size=64, output_dir="outputs/tiles"):
    """
    큰 이미지를 tile_size x tile_size 크기의 타일로 분할합니다.
    
    Parameters:
        image_path: 원본 이미지 경로
        tile_size: 타일 크기 (기본 64px)
        output_dir: 저장할 폴더
    """
    img = PILImage.open(image_path)
    w, h = img.size
    
    os.makedirs(output_dir, exist_ok=True)
    
    tiles = []
    cols = w // tile_size
    rows = h // tile_size
    
    print(f"이미지 크기: {w}x{h}")
    print(f"타일 크기: {tile_size}x{tile_size}")
    print(f"생성될 타일 수: {cols} x {rows} = {cols * rows}장")
    print()
    
    for row in range(rows):
        for col in range(cols):
            # 각 타일 영역 계산
            left = col * tile_size
            top = row * tile_size
            right = left + tile_size
            bottom = top + tile_size
            
            tile = img.crop((left, top, right, bottom))
            
            # 저장
            filename = f"tile_{row}_{col}.png"
            tile.save(os.path.join(output_dir, filename))
            tiles.append(tile)
    
    print(f"{len(tiles)}개의 타일이 {output_dir}/ 에 저장되었습니다!")
    return tiles

# 실행 예시 (생성된 이미지가 있을 때)
sample_path = "outputs/dalle_office_desk.png"
if os.path.exists(sample_path):
    tiles = split_into_tiles(sample_path, tile_size=64)
    
    # 처음 16개 타일 미리보기
    preview_count = min(16, len(tiles))
    cols = 4
    rows = (preview_count + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(8, 8))
    for i, ax in enumerate(axes.flat):
        if i < preview_count:
            ax.imshow(tiles[i].resize((128, 128), PILImage.Resampling.NEAREST))
            ax.set_title(f"tile_{i // cols}_{i % cols}", fontsize=9)
        ax.axis("off")
    plt.suptitle("분할된 타일 미리보기 (확대)", fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print(f"{sample_path} 파일이 없습니다. 위의 DALL-E 셀을 먼저 실행하세요!")

### 5-4. 게임 프로젝트에 에셋 적용하기

가공이 끝난 이미지를 게임 프로젝트의 에셋 폴더에 복사하면 됩니다.

```
CG Town 프로젝트 에셋 경로:
frontend/public/images/
├── tiles/          ← 타일 이미지
├── charactor/      ← 캐릭터 스프라이트
└── objects/         ← 오브젝트 이미지
```

In [ ]:
import shutil
import os

# 프로젝트의 에셋 경로 (이 노트북 기준 상대 경로)
PROJECT_ASSETS = "../frontend/public/images"

def copy_to_project(source_path, asset_type="objects"):
    """
    생성한 에셋을 게임 프로젝트 폴더로 복사합니다.
    
    asset_type: "tiles", "objects", "charactor" 중 하나
    """
    dest_dir = os.path.join(PROJECT_ASSETS, asset_type)
    os.makedirs(dest_dir, exist_ok=True)
    
    filename = os.path.basename(source_path)
    dest_path = os.path.join(dest_dir, filename)
    
    shutil.copy2(source_path, dest_path)
    print(f"복사 완료: {source_path} → {dest_path}")

# 사용 예시 (파일이 있을 때)
if os.path.exists("outputs/desk_64x64.png"):
    copy_to_project("outputs/desk_64x64.png", asset_type="objects")
else:
    print("복사할 파일이 없습니다. 위의 셀들을 먼저 실행하세요!")
    print(f"\n참고: 프로젝트 에셋 경로 = {os.path.abspath(PROJECT_ASSETS)}")

---
## 6. 실습 과제

아래 과제를 직접 해봅시다!

### 과제 1: 사무실 오브젝트 3개 생성하기

DALL-E 또는 Stability AI를 사용해서 아래 오브젝트를 만들어 보세요:
1. 사무실 의자
2. 화분 (식물)
3. 복사기 or 프린터

각각 64x64로 리사이즈해서 `outputs/` 폴더에 저장하세요.

In [ ]:
# 과제 1: 여기에 코드를 작성하세요!
# 힌트: 위에서 배운 client.images.generate() 를 사용하면 됩니다

objects_to_create = [
    {"name": "office_chair", "prompt": "여기에 프롬프트를 적으세요"},
    {"name": "plant", "prompt": "여기에 프롬프트를 적으세요"},
    {"name": "printer", "prompt": "여기에 프롬프트를 적으세요"},
]

# for obj in objects_to_create:
#     response = client.images.generate(...)
#     ...
#     print(f"{obj['name']} 생성 완료!")

### 과제 2: 프롬프트 실험

같은 대상(예: 책상)에 대해 프롬프트를 다르게 써보고 결과를 비교해 보세요.

예시:
- 프롬프트 A: `"office desk, top view"`
- 프롬프트 B: `"pixel art office desk with monitor and keyboard, top-down view, 2D game asset, 16-bit retro style"`

어떤 프롬프트가 더 좋은 결과를 만드나요?

In [ ]:
# 과제 2: 여기에 코드를 작성하세요!

prompt_a = "office desk, top view"
prompt_b = "pixel art office desk with monitor and keyboard, top-down view, 2D game asset, 16-bit retro style"

# 두 프롬프트로 각각 이미지를 생성하고 비교해 보세요
# ...

---
## 요약

오늘 배운 것:

| 단계 | 내용 | 사용한 도구 |
|------|------|-------------|
| 1 | 생성형 AI 개념 | - |
| 2 | API 호출 기초 | `requests`, `.env` |
| 3 | DALL-E 이미지 생성 | `openai` 라이브러리 |
| 4 | Stability AI 이미지 생성 | `requests` (REST API) |
| 5 | 이미지 가공 (리사이즈, 크롭, 분할) | `Pillow` |

### 다음 단계
- 생성한 에셋으로 게임 맵 꾸미기
- 스프라이트시트 만들기 (여러 프레임 → 애니메이션)
- 더 나은 프롬프트 기법 연구